In [1]:
!pip install transformers datasets scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 9.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "

In [6]:
import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW
from transformers import (
    RobertaTokenizerFast, RobertaForSequenceClassification,
    get_scheduler, DataCollatorWithPadding
)
from datasets import load_dataset
from sklearn.metrics import accuracy_score, f1_score
import numpy as np
from tqdm import tqdm


In [3]:
dataset = load_dataset("imdb")
dataset = dataset.remove_columns([col for col in dataset["train"].column_names if col not in ["text", "label"]])


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [4]:
tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding=True, truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

# Speed boost with subset (optional: remove `.select` for full dataset)
train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(3000))  # faster training
test_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(500))    # faster evaluation


data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=data_collator)
test_loader = DataLoader(test_dataset, batch_size=32, collate_fn=data_collator)


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [5]:
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


NameError: name 'RobertaForSequenceClassification' is not defined

In [6]:
optimizer_grouped_parameters = [
    {"params": model.roberta.embeddings.parameters(), "lr": 5e-5},
    {"params": model.roberta.encoder.layer[:6].parameters(), "lr": 3e-5},
    {"params": model.roberta.encoder.layer[6:].parameters(), "lr": 1e-5},
    {"params": model.classifier.parameters(), "lr": 1e-4}
]

optimizer = AdamW(optimizer_grouped_parameters, weight_decay=0.01)
num_training_steps = len(train_loader) * 4  # 4 epochs
lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)


In [2]:
def train():
    model.train()
    for epoch in range(4):
        loop = tqdm(train_loader, leave=True)
        for batch in loop:
            batch = {k: v.to(device) for k, v in batch.items() if k in ["input_ids", "attention_mask", "label"]}
            outputs = model(**batch)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            loop.set_description(f"Epoch {epoch+1}")
            loop.set_postfix(loss=loss.item())


In [3]:
def evaluate():
    model.eval()
    predictions, labels, texts = [], [], []
    label_map = {0: "Negative", 1: "Positive"}  # IMDb has 2 classes

    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            label_ids = batch["label"].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=label_ids)
            logits = outputs.logits
            preds = torch.argmax(logits, axis=-1)
            predictions.extend(preds.cpu().numpy())
            labels.extend(label_ids.cpu().numpy())
            texts.extend(tokenizer.batch_decode(input_ids, skip_special_tokens=True))
            if len(texts) >= 2:  # Show only 2 predictions
                break

    print("\n📌 Sample Predictions (2 only):")
    for text, pred, label in zip(texts[:2], predictions[:2], labels[:2]):
        print(f"\n📝 Text: {text[:120]}...")
        print(f"🔮 Predicted Label: {label_map[pred]} | 🎯 Actual Label: {label_map[label]}")

    acc = accuracy_score(labels, predictions)
    error_rate = 1 - acc
    f1 = f1_score(labels, predictions, average="weighted")
    print(f"\n✅ Accuracy    : {acc*100:.2f}%")
    print(f"❌ Error Rate : {error_rate*100:.2f}%")
    print(f"📊 F1 Score   : {f1*100:.2f}%")


In [4]:
train()
evaluate()


NameError: name 'model' is not defined